In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

In [4]:
# Dataset path
ecg_first_30_users_file_path = "../../data/processed/dataset_ecg_first_30_users.csv"

In [5]:
# Read the dataset
df_ecg = pd.read_csv(ecg_first_30_users_file_path)

In [6]:
# Explore the dataset
df_ecg.head()

,User,Condition,Timestamp,Column2,Column3,Column4,Column5,Column6
0,user0,user_cognitive-fatigue,15:06:28,72,0.849609,3,42,870
1,user0,user_cognitive-fatigue,15:06:29,72,0.751953,3,2,770
2,user0,user_cognitive-fatigue,15:06:30,73,0.766602,3,b,785
3,user0,user_cognitive-fatigue,15:06:31,73,0.752930,3,3,771
4,user0,user_cognitive-fatigue,15:06:32,74,0.783203,3,16,802
